# 2 : show glacier evolution dependent on the applied climate change

In [1]:
# download it here https://cluster.klima.uni-bremen.de/~lschuster/glacierMIP3_analysis/glacierMIP3_mar14_models_all_rgi_regions_sum_scaled.nc
# and change the path to your local path
path_merged_runs_scaled = '/home/www/lschuster/glacierMIP3_analysis/glacierMIP3_mar14_models_all_rgi_regions_sum_scaled.nc'

In [2]:
import xarray as xr
import numpy as np
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
from datetime import date
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from help_func import pal_models, model_order, d_reg_num_name
hue_order = model_order
ds_reg_models = xr.open_dataset(path_merged_runs_scaled)

IDEAS:
- for every region, all 80 experiments as lines dependent on the warming scenario (blue to red), either with global warming or regional warming 

### Random code...

In [ ]:
pd_global_temp_exp_gcm = pd_global_temp_exp.loc[pd_global_temp_exp.gcm =='gfdl-esm4']

pd_global_temp_exp_gcm.index = pd_global_temp_exp_gcm.period_scenario

ds_reg_models_med_v_glob = ds_reg_models_med_v.sum(dim='rgi_reg')

# this is really unefficient
ds_reg_models_med_v_glob.to_dataframe().reset_index().pivot_table(values = 'volume_m3', index=)

In [4]:
# --> comes from isimip3b_postprocessing_analysis/isimip3b_postprocess_to_monthly.ipynb
pd_global_temp_exp = pd.read_csv('../data/temp_ch_ipcc_isimip3b.csv', index_col = 0)

pd_reg_models_med_v_glob =  ds_reg_models_med_v.sum(dim='rgi_reg').to_dataframe().reset_index().groupby(['period_scenario', 'simulation_year', 'gcm']).sum().reset_index()

pd_v_l = []
for gcm in ds_reg_models.gcm:
    pd_global_temp_exp_gcm = pd_global_temp_exp.loc[pd_global_temp_exp.gcm == 'gcm']
    ds_reg_models_med_v = ds_reg_models.volume_m3.median(dim='model_author').sel(gcm=gcm).sum(dim='rgi_reg')

    ds_reg_models_med_v = ds_reg_models_med_v.assign_coords(temp_ch_ipcc = ('period_ssp',
                                                                            pd_global_temp_exp_gcm.loc[pd_reg_models_med_v_glob.period_scenario, 'temp_ch_ipcc'].values))
    
    pd_v = ds_reg_models_med_v.to_dataframe().reset_index()
    pd_v['gcm'] = gcm.values
    pd_v_l.append(pd_v)
pd_glob_models_med_v = pd.concat(pd_v_l)

ds = pd_glob_models_med_v.groupby(['simulation_year','period_scenario', 'gcm']).mean().to_xarray()

d2000 = pd_glob_models_med_v.loc[pd_glob_models_med_v.simulation_year == 2000]

,gcm,period_scenario,temp_ch_ipcc
0,gfdl-esm4,1851-1870_hist,0.171409
1,gfdl-esm4,1901-1920_hist,0.418289
2,gfdl-esm4,1951-1970_hist,0.332281
3,gfdl-esm4,1995-2014_hist,0.841467
4,gfdl-esm4,2021-2040_ssp126,1.433792
...,...,...,...
75,ukesm1-0-ll,2081-2100_ssp370,5.780495
76,ukesm1-0-ll,2021-2040_ssp585,2.259733
77,ukesm1-0-ll,2041-2060_ssp585,3.586968
78,ukesm1-0-ll,2061-2080_ssp585,5.170544


In [ ]:
plt.plot(ds_reg_models_med_v.simulation_year, ds_reg_models_med_v.volume_m3.sum(dim='rgi_reg'));

In [ ]:

sns.scatterplot(data = pd_glob_models_med_v, x = 'simulation_year', hue='temp_ch_ipcc',
             y='volume_m3', palette='viridis');


sns.lineplot(data = d2000, x = 'temp_ch_ipcc',
             y='volume_m3', palette='viridis');